## Hindi ULMFiT from scratch

In [1]:
!nvidia-smi

Wed Oct 28 11:37:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *

from fastai.text import *

In [3]:
bs = 128

In [4]:
torch.cuda.set_device(0)

In [5]:
data_path = Config.data_path(); data_path

PosixPath('/home/jupyter/.fastai/data')

This will create a `hiwiki` folder, containing a `hiwiki` file with the wikipedia contents.(For other languages replace `hi` with the appropriate code from [list of wikipedias](https://meta.wikimedia.org/wiki/List_of_Wikipedias))

In [6]:
data_path = Config.data_path()
lang = 'hi'
name = f'{lang}wiki'
path = data_path/name
dest = path/'docs'
lm_fns = [f'{lang}_wt_bwd', f'{lang}_wt_vocab_bwd']

In [7]:
path.ls()

[PosixPath('/home/jupyter/.fastai/data/hiwiki/hiwiki'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/hi_textlist_class'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/log'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/hi_textlist_class_product_reviews'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/docs'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/models'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/hiwiki-latest-pages-articles.xml'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/hi_textlist_class_bwd'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/hiwiki-latest-pages-articles.xml.bz2'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/hi_textlist_class_movie_reviews')]

### Hindi Wikipedia Model

### Create Pretrained language model

In [22]:
%%time
data = (TextList.from_folder(dest)
       .split_by_rand_pct(0.1, seed=42)
       .label_for_lm()
       .databunch(bs=bs, num_workers=1, backwards=True))
data.save(f'{lang}_databunch_bwd')
len(data.vocab.itos), len(data.train_ds)

CPU times: user 22.5 s, sys: 4.84 s, total: 27.4 s
Wall time: 7min 42s


(60000, 17222)

In [56]:
data = load_data(dest, f'{lang}_databunch_bwd', bs=bs, backwards=True)

In [57]:
data.show_batch()

idx,text
0,"रस xxunk उपर्युक्त लिए के जमाने जैली अच्छी \n \n । जाये हो प्राप्त रस का पेक्टिन स्वच्छ कि जिससे लीजिए xxunk से विधि xxunk सुबह और xxunk में वातावरण शीतल भर रात लिए के xxunk के रस । चाहिए लेना छान से कपड़े मोटे को रस हुए उबाले इस । है होती उपलब्ध में मात्रा ज्यादा पेक्टिन हमें और है जाता हो परिवर्तित में पेक्टिन , xxunk से"
1,", विलायक बहुमूल्य अनेक इनसे । हैं हुए सिद्ध के महत्व बड़े से दृष्टि औद्योगिक व्युत्पंन अनेक ऐसे । है बनाता व्युत्पन्न से सरलता हा संयुक्त से अभिकारकों अनेक और है होता सक्रिय अधिक हाइड्रोकार्बन असतृप्त । हैं सल्फेट हाइड्रोजन तथा फास्फेट हाइड्रोजन , नाइट्राइट , नाइट्राइट , नाइट्रेट , मरकैप्टन , ऐमिन , ऐल्कॉक्साइड सोडियम , ऐल्कोहाल , आयोडाइड , ब्रोमाइड , क्लोराइड व्युत्पन्न ऐसे । हैं उपयोग विविध"
2,"3 xxunk . 3 कोल्हापुर . 2 सातारा . 1 \n \n -कळंब : देवक \n xxunk : गोत्र \n -सूर्य : वंश \n xxunk - : . 96 \n -पंचपल्लव : देवक \n xxunk : गोत्र \n -चंद्र : वंश \n xxunk - : . 95 \n xxunk , -पंचपल्लव : देवक \n xxunk : गोत्र \n -सूर्य"
3,प्रति के अभिनय उन्हें और मिला रास्ता का कैरियर अपने को कपूर ही में दिशानिर्देश और संरक्षण के हुसैन रज़ा आमिर । की अदा भूमिका मुख्य इसमें उन्होंने में क्रम इस और दें ऑडिशन उसमें था निर्माण नाटक वार्षिक का स्कूल जो आंट xxunk वह कि दिया आदेश भरा चुनौती उन्हें ने देव तरुण कैप्टन हेड उनके जब हुआ पर यहीं से ढंग नाटकीय बड़े परिचय उनका से अभिनय बल्कि
4,करते शुरू से गणना की xxunk लिए के मानों छोटे हम में विधि इस \n \n । है गया दिया कर भी ) xxunk ( xxunk उन्हें है गयी की गणना की मानों जिन में तकनीक इस \n \n \n । लगेगा समय अधिक बहुत इसमें कारण इस । पड़ेगी करनी बार अनेक भी और गणना ही एक तो जाय लिया उदाहरण का संख्या बड़ी बजाय के ५


In [58]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [59]:
lr = 1e-2
lr *= bs/48 # scale LR by batch size

In [61]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.534879,4.564542,0.257597,20:04
1,4.477128,4.556511,0.254814,20:04
2,4.525117,4.537685,0.256364,20:04
3,4.462911,4.468653,0.261098,20:04
4,4.402483,4.398094,0.264974,20:04
5,4.312982,4.315479,0.270799,20:05
6,4.216850,4.221115,0.277799,20:05
7,4.104349,4.129683,0.285398,20:04
8,4.022043,4.072096,0.290785,20:03
9,3.918710,4.060677,0.292471,20:04


Save the pretrained model and vocab:

In [60]:
lm_fns

['hi_wt_bwd', 'hi_wt_vocab_bwd']

In [62]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

In [63]:
(path/'models').ls()

[PosixPath('/home/jupyter/.fastai/data/hiwiki/models/20epoch_hi_vocab.pkl'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/models/hifine_tuned_enc_movie_reviews.pth'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/models/hifine_tuned_enc.pth'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/models/hiclas_product_reviews.pth'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/models/hi_wt.pth'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/models/hiclas.pth'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/models/hifine_tuned_movie_reviews.pth'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/models/hi_wt_vocab_bwd.pkl'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/models/hifine_tuned_product_reviews.pth'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/models/hi_wt_bwd.pth'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/models/20epoch_hi_wt.pth'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/models/hifine_tuned_enc_product_reviews.pth'),
 PosixPath('/home/jupyter/.fastai/data/hiwiki/mode

In [64]:
# learn.model.load_state_dict(torch.load(path/'models/20epoch_hi_wt.pth'))

In [65]:
# learn.model.load_state_dict(torch.load(path/'models/hi_wt.pth'))

In [66]:
# learn.load(path/'models/hi_wt');
# loading a saved model:https://forums.fast.ai/t/load-model-from-pth-file/37440

In [20]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(60000, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(60000, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=60000, bias=True)
    (output_dp): RNNDropout()
  )
)

### Hindi Sentiment Analysis

## BBC Articles(Hindi)

#### Language Model

We are gonna perform sentiment analysis on 2 datasets: BBC news articles,IIT-P movie reviews and IIT-P product reviews. The datasets can be found [here](https://github.com/AI4Bharat/indicnlp_corpus#publicly-available-classification-datasets).

In [8]:
train_df = pd.read_csv('data/bbc-articles-hi/hi-train.csv', header=None)
train_df.head()

,0,1
0,india,मेट्रो की इस लाइन के चलने से दक्षिणी दिल्ली से...
1,pakistan,नेटिजन यानि इंटरनेट पर सक्रिय नागरिक अब ट्विटर...
2,news,इसमें एक फ़्लाइट एटेनडेंट की मदद की गुहार है औ...
3,india,"प्रतीक खुलेपन का, आज़ाद ख्याली का और भीड़ से अ..."
4,india,ख़ासकर पिछले 10 साल तक प्रधानमंत्री रहे मनमोहन...


In [9]:
test_df = pd.read_csv('data/bbc-articles-hi/hi-test.csv', header=None)
test_df.head()

,0,1
0,india,बुधवार को राज्य सभा में विपक्ष के सवालों के जव...
1,india,लखनऊ स्थित पत्रकार समीरात्मज मिश्र को बुलंदशहर...
2,india,लगभग 1300 हेक्टेयर ज़मीन का अधिग्रहण किया जा च...
3,international,हालांकि उनके अंगरक्षकों को बमों को जाम करने वा...
4,india,आयोग का कहना है कि इस तरह के परीक्षण से महिलाओ...


In [10]:
df = pd.concat([train_df,test_df], sort=False)

In [11]:
data_lm = (TextList.from_df(df, path, cols=1)
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1, backwards=True))

In [71]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns,  config={**awd_lstm_lm_config, 'n_hid': 1152},
                                   drop_mult=1.0)

In [72]:
lr = 1e-3
lr *= bs/48

In [73]:
# finetuning the language model
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.427934,4.046300,0.275558,01:06
1,4.109862,3.944758,0.284251,01:06


In [74]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.917224,3.920418,0.286749,01:07
1,3.857886,3.882085,0.291003,01:07
2,3.809151,3.856278,0.293334,01:06
3,3.736108,3.836730,0.296154,01:06
4,3.696278,3.824736,0.297085,01:06
5,3.627378,3.819150,0.297935,01:06
6,3.586017,3.818778,0.298489,01:07
7,3.565432,3.818451,0.298695,01:06


In [75]:
learn_lm.save(f'{lang}fine_tuned_bwd')
learn_lm.save_encoder(f'{lang}fine_tuned_enc_bwd')

In [76]:
learn_lm = None
learn = None
gc.collect()

100

### Classifier

In [12]:
data_clas = (TextList.from_df(train_df, path, vocab=data_lm.vocab, cols=1)
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols=0)
    .databunch(bs=bs, num_workers=1, backwards=True))

data_clas.save(f'{lang}_textlist_class_bwd')

In [78]:
data_clas = load_data(path, f'{lang}_textlist_class_bwd', bs=bs, num_workers=1, backwards=True)

In [79]:
data_clas.classes, data_clas.c

(['business',
  'china',
  'entertainment',
  'india',
  'institutional',
  'international',
  'learningenglish',
  'multimedia',
  'news',
  'pakistan',
  'science',
  'social',
  'southasia',
  'sport'],
 14)

In [13]:
from sklearn.metrics import f1_score

@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1), average='micro')

mcc = MatthewsCorreff()

In [14]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1, mcc]).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc_bwd')
learn_c.freeze()

In [82]:
lr=2e-2
lr *= bs/48

In [83]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,1.252077,1.072506,0.682081,0.682081,0.563318,00:23
1,0.951820,0.857328,0.710983,0.708093,0.615012,00:22


In [84]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.788937,0.756339,0.751445,0.751445,0.673780,00:23
1,0.716446,0.625376,0.791907,0.791907,0.720667,00:22


In [85]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.739347,0.806878,0.765896,0.765896,0.684704,00:26
1,0.596376,0.655883,0.812139,0.812139,0.752108,00:24


In [86]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.438991,0.711892,0.780347,0.780347,0.704294,00:38
1,0.358268,0.628176,0.823699,0.823699,0.766901,00:39


In [87]:
learn_c.unfreeze()
learn_c.fit_one_cycle(2, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.229670,0.622310,0.809249,0.809249,0.745110,00:46
1,0.188292,0.629452,0.826590,0.826590,0.768872,00:44


In [88]:
learn_c.save(f'{lang}clas_bwd')

In [15]:
learn_c.load(f'{lang}clas_bwd');

In [16]:
data_valid = (TextList.from_df(test_df, path, vocab=data_lm.vocab, cols=1)
    .no_split()
    .label_from_df(cols=0)
    .databunch(bs=bs, num_workers=1))

/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py:183: UserWarning: `no_split` is deprecated, please use `split_none`.
  warn("`no_split` is deprecated, please use `split_none`.")


In [17]:
data_valid

TextClasDataBunch;

Train: LabelList (866 items)
x: TextList
xxbos बुधवार को राज्य सभा में विपक्ष के सवालों के जवाब में मनमोहन सिंह ने कहा , " अगर किसी तरह की गड़बड़ी पाई जाती है , तो हम कड़ी से कड़ी कार्रवाई करेंगे . "मंगलवार को संसद में भारत के नियंत्रक और महालेखा परीक्षक की रिपोर्ट पेश की गई है . किसानों के क़र्ज़ की माफ़ी स्कीम पर तैयार इस रिपोर्ट में कहा गया है कि पिछले वित्तीय वर्ष में जितने भी लोगों के क़र्ज़ बैंकों ने माफ़ किए थे , उनमें से 22 फ़ीसद मामले फर्ज़ी थे . रिपोर्ट में कहा गया है कि बैंक के कर्मचारियों ने फर्ज़ी खातों में कर्ज़ माफ़ किए हैं और वो सारी राशि ख़ुद xxunk ली है . विपक्ष ने मामले पर सरकार को घेरना शुरू कर दिया है . मुख्य विपक्षी दल भारतीय जनता पार्टी के प्रवक्ता प्रकाश जावडेकर ने कहा है कि इस तरह की गड़बड़ी इतने बड़े पैमाने पर सिर्फ़ तकनीकी कमियों की वजह से नहीं हो सकती है . विपक्ष और सरकार को समर्थन दे रहे xxunk जैसे समाजवादी पार्टी मामले को संसद में भी उठा रही है . लोकसभा में विपक्ष की नेता सुषमा स्वराज ने अध्यक्ष से मामले पर बहस करवाने की मांग की है . मह

In [18]:
%%time
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(test_df[1]), 'actual_label': list(test_df[0]), 'predicted_label': ['']*test_df.shape[0]}
all_nodes = list(set(train_df[0]))
for node in all_nodes:
    df_dict[node] = ['']*test_df.shape[0]
    
i2c = {}
for key, value in learn_c.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
for index, row in df_result.iterrows():
    pred = learn_c.predict(data_valid.train_ds[index])
    for node in all_nodes:
        row[node] = pred[2][learn_c.data.c2i[node]].item()
    row['predicted_label'] = i2c[pred[0].data.item()]
df_result.head()

CPU times: user 2min, sys: 12.6 s, total: 2min 12s
Wall time: 1min 59s


,query,actual_label,predicted_label,institutional,sport,southasia,international,multimedia,china,science,social,business,entertainment,news,india,pakistan,learningenglish
0,बुधवार को राज्य सभा में विपक्ष के सवालों के जव...,india,india,9.90869e-07,2.25081e-05,1.89537e-05,5.9292e-06,9.41948e-06,1.93653e-05,9.62404e-06,9.56639e-07,0.000185502,1.38107e-06,0.000185853,0.999535,4.39761e-06,1.20206e-07
1,लखनऊ स्थित पत्रकार समीरात्मज मिश्र को बुलंदशहर...,india,india,1.07988e-08,8.56306e-06,6.10689e-07,6.73537e-05,6.15479e-07,4.61804e-08,4.06867e-06,9.79698e-07,1.44182e-08,1.46189e-07,1.9332e-06,0.999916,3.981e-08,1.97786e-10
2,लगभग 1300 हेक्टेयर ज़मीन का अधिग्रहण किया जा च...,india,india,1.06525e-07,4.28014e-06,6.64218e-06,2.29358e-05,6.75997e-06,4.85952e-06,7.12708e-05,2.13962e-06,1.52338e-05,1.88085e-06,5.85685e-05,0.999804,8.91926e-07,2.28584e-08
3,हालांकि उनके अंगरक्षकों को बमों को जाम करने वा...,international,international,3.38236e-05,6.26285e-06,1.60216e-06,0.999593,0.000213016,5.12082e-07,1.76678e-05,1.12523e-05,2.51526e-07,4.85849e-06,6.43646e-06,9.86394e-05,1.43315e-07,1.23582e-05
4,आयोग का कहना है कि इस तरह के परीक्षण से महिलाओ...,india,international,9.22305e-06,0.000115362,2.78595e-05,0.992164,0.000121846,1.85947e-05,6.24156e-05,0.00110607,4.99344e-08,3.59123e-05,4.82238e-06,0.00631702,1.00115e-05,7.23926e-06


In [19]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 17 columns):
query              866 non-null object
actual_label       866 non-null object
predicted_label    866 non-null object
institutional      866 non-null object
sport              866 non-null object
southasia          866 non-null object
international      866 non-null object
multimedia         866 non-null object
china              866 non-null object
science            866 non-null object
social             866 non-null object
business           866 non-null object
entertainment      866 non-null object
news               866 non-null object
india              866 non-null object
pakistan           866 non-null object
learningenglish    866 non-null object
dtypes: object(17)
memory usage: 115.1+ KB


In [20]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.7875288683602771

In [21]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.7114524040323768

In [22]:
f1_score(df_result['actual_label'], df_result['predicted_label'], average='micro')

0.7875288683602771

Similary, create a bwd model for the other two datasets.

## IIT-P Product Reviews

In [23]:
train_df = pd.read_csv('data/iitp-product-reviews/hi-train.csv', header=None)
train_df.head()

,0,1
0,negative,एंडराॅयड के मामले में यह थोड़ा पीछे है ।
1,neutral,यह एस्पेक्ट रेशो का ईश्यू है और हम आशा करते है...
2,positive,लेकिन इस तरह के एक मॉडल के एक घर कंप्यूटर के ल...
3,positive,गिर वन राष्ट्रीय उद्यान बाघ संरक्षित क्षेत्र ह...
4,neutral,"और हां , इस फिल्म में हर किरदार भारद्वाज को भर..."


In [24]:
valid_df = pd.read_csv('data/iitp-product-reviews/hi-valid.csv', header=None)
valid_df.head()

,0,1
0,positive,"एक ऐसा भी जानवर है , जिसे आप पसंद तो करते होंग..."
1,neutral,इतना नहीं बल्कि यहां पर कंपनी अपना पहला फाइव स...
2,neutral,"डिजिफलीप प्रो XT811 , 1200 x 800 P और 8 इंच डि..."
3,positive,निकोन डी810 कैमरे की सबसे खास बात ये है कि इसम...
4,positive,"इसमें विंडोज 8 64-बिट ऑपरेटिंग सिस्टम है , लेक..."


In [25]:
test_df = pd.read_csv('data/iitp-product-reviews/hi-test.csv', header=None)
test_df.head()

,0,1
0,neutral,"मगर , इस तरफ कोई ध्यान नहीं दिया जा रहा ।"
1,negative,गेम कई बार मुश्किल मालूम देता है ।
2,negative,मल्टीप्लेयर में छोटी स्टोरीलाइन है ।
3,positive,हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा बेहतरीन...
4,positive,कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमर...


In [26]:
df = pd.concat([train_df,test_df, valid_df], sort=False)

In [27]:
data_lm = (TextList.from_df(df, path, cols=1)
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1, backwards=True))

In [28]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [29]:
lr = 1e-3
lr *= bs/48

In [30]:
# finetuning the language model
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.950140,4.135259,0.288170,00:02
1,4.510882,3.899812,0.299944,00:02


In [31]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.029308,3.835566,0.304074,00:02
1,3.966516,3.714226,0.322489,00:02
2,3.890238,3.638484,0.328292,00:02
3,3.820797,3.592114,0.331696,00:02
4,3.762119,3.564978,0.331752,00:02
5,3.723857,3.551371,0.331473,00:02
6,3.692766,3.544807,0.331250,00:02
7,3.655950,3.546231,0.331808,00:02


In [32]:
learn_lm.save(f'{lang}fine_tuned_product_reviews_bwd')
learn_lm.save_encoder(f'{lang}fine_tuned_enc_product_reviews_bwd')

In [33]:
learn_lm = None
learn = None
gc.collect()

100

### Classifier

In [34]:
product_reviews = pd.concat([train_df,valid_df], sort=False)

In [35]:
data_clas = (TextList.from_df(product_reviews, path, vocab=data_lm.vocab, cols=1)
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols=0)
    .databunch(bs=bs, num_workers=1, backwards=True))

data_clas.save(f'{lang}_textlist_class_product_reviews_bwd')

In [36]:
data_clas = load_data(path, f'{lang}_textlist_class_product_reviews_bwd', bs=bs, num_workers=1, backwards=True)

In [37]:
data_clas.classes, data_clas.c

(['negative', 'neutral', 'positive'], 3)

In [38]:
from sklearn.metrics import f1_score

@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1), average='micro')

mcc = MatthewsCorreff()

In [39]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1, mcc]).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc_product_reviews_bwd')
learn_c.freeze()

In [40]:
lr=2e-2
lr *= bs/48

In [41]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.881692,0.851152,0.548936,0.548936,0.241555,00:01
1,0.797685,0.697554,0.693617,0.693617,0.475431,00:01


In [42]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.775430,0.737605,0.661702,0.661702,0.420812,00:01
1,0.741524,0.790249,0.651064,0.651064,0.418962,00:01


In [43]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.746863,0.775865,0.672340,0.672340,0.442657,00:01
1,0.640803,0.648860,0.719149,0.719149,0.531514,00:01


In [44]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.524273,0.788469,0.676596,0.676596,0.474974,00:01
1,0.433442,0.678182,0.727660,0.727660,0.546638,00:01


In [45]:
learn_c.unfreeze()
learn_c.fit_one_cycle(2, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.282072,0.729670,0.731915,0.731915,0.552680,00:02
1,0.265053,0.768575,0.731915,0.731915,0.550103,00:02


In [46]:
learn_c.save(f'{lang}clas_product_reviews_bwd')

In [47]:
learn_c.load(f'{lang}clas_product_reviews_bwd');

In [48]:
data_valid = (TextList.from_df(test_df, path, vocab=data_lm.vocab, cols=1)
    .split_none()
    .label_from_df(cols=0)
    .databunch(bs=bs, num_workers=1, backwards=True))

In [49]:
data_valid

TextClasDataBunch;

Train: LabelList (523 items)
x: TextList
xxbos मगर , इस तरफ कोई ध्यान नहीं दिया जा रहा ।,xxbos गेम कई बार मुश्किल मालूम देता है ।,xxbos xxunk में छोटी xxunk है ।,xxbos हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा बेहतरीन होती है – अत्याधुनिक और xxunk लुक जो हर किसी को xxunk हैं ।,xxbos कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमरों में xxunk करते हुए इस सीरीज का सबसे शानदार कैमरा लेकर आई है ।
y: CategoryList
neutral,negative,negative,positive,positive
Path: /home/jupyter/.fastai/data/hiwiki;

Valid: LabelList (0 items)
x: TextList

y: CategoryList

Path: /home/jupyter/.fastai/data/hiwiki;

Test: None

In [50]:
%%time
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(test_df[1]), 'actual_label': list(test_df[0]), 'predicted_label': ['']*test_df.shape[0]}
all_nodes = list(set(train_df[0]))
for node in all_nodes:
    df_dict[node] = ['']*test_df.shape[0]
    
i2c = {}
for key, value in learn_c.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
for index, row in df_result.iterrows():
    pred = learn_c.predict(data_valid.train_ds[index])
    for node in all_nodes:
        row[node] = pred[2][learn_c.data.c2i[node]].item()
    row['predicted_label'] = i2c[pred[0].data.item()]
df_result.head()

CPU times: user 26.1 s, sys: 116 ms, total: 26.2 s
Wall time: 25.7 s


,query,actual_label,predicted_label,positive,neutral,negative
0,"मगर , इस तरफ कोई ध्यान नहीं दिया जा रहा ।",neutral,neutral,0.00866491,0.514536,0.476799
1,गेम कई बार मुश्किल मालूम देता है ।,negative,positive,0.984325,0.00596845,0.00970669
2,मल्टीप्लेयर में छोटी स्टोरीलाइन है ।,negative,negative,0.211786,0.11343,0.674783
3,हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा बेहतरीन...,positive,positive,0.998921,0.00069841,0.000380461
4,कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमर...,positive,positive,0.991544,0.00837572,8.04066e-05


In [51]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523 entries, 0 to 522
Data columns (total 6 columns):
query              523 non-null object
actual_label       523 non-null object
predicted_label    523 non-null object
positive           523 non-null object
neutral            523 non-null object
negative           523 non-null object
dtypes: object(6)
memory usage: 24.6+ KB


In [52]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.7590822179732314

In [53]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.5982714991368793

In [54]:
f1_score(df_result['actual_label'], df_result['predicted_label'], average='micro')

0.7590822179732314

## IIT-P movie reviews

In [55]:
train_df = pd.read_csv('data/iit-p-movie-reviews/hi-train.csv', header=None)
train_df.head()

,0,1
0,neutral,"निर्माता :\nशीतल विनोद तलवार, मधु‍ मैंटेना\n\n..."
1,positive,’उड़ान’ से विक्रमादित्य\nमोटवाने\nने अच्छे सिन...
2,neutral,फिल्म में गानों के दृश्य में अनुष्का को माइक क...
3,neutral,फांसी चढ़ने से पहले वह पाकिस्तान के सदर से खास...
4,positive,राज कुमार हिरानी ने इस मूवी में भी अपने डायरेक...


In [56]:
valid_df = pd.read_csv('data/iit-p-movie-reviews/hi-valid.csv', header=None)
valid_df.head()

,0,1
0,negative,"ये फगली-फगली क्या है, ये फगली-फगली? इस बात का ..."
1,neutral,निर्माता :\nआदित्य चोपड़ा\n\nनिर्देशक :\nहबीब ...
2,positive,इस कसौटी पर करण काफी हद तक खरे भी उतरे हैं।
3,positive,"छोटे शहर की गालियां, छतें और विहंगम दृश्य मोहक..."
4,positive,मनीष और जिशान का काम भी लोगों को याद रहेगा।


In [57]:
test_df = pd.read_csv('data/iit-p-movie-reviews/hi-test.csv', header=None)
test_df.head()

,0,1
0,negative,काव्या अपनी खुन्नस और जिद में सिद्धार्थ को तबा...
1,negative,"निर्माता :\nफरहान अख्तर, रितेश सिधवानी\n\nनिर्..."
2,positive,असमर्पित रिश्ते में यकीन रखने वाले दोनों मौज-म...
3,neutral,"बेवकूफियां को हबीब फैज़ल ने लिखा है, जिनका नाम..."
4,neutral,तब वह मोटी थी और जेरी ने उस पर ध्यान नहीं दिय...


In [58]:
df = pd.concat([train_df,test_df, valid_df], sort=False)

In [59]:
data_lm = (TextList.from_df(df, path, cols=1)
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1, backwards=True))

In [60]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [61]:
lr = 1e-3
lr *= bs/48

In [62]:
# finetuning the language model
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.504690,3.950453,0.294615,00:16
1,4.103548,3.837047,0.305636,00:16


In [63]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.865649,3.805204,0.308943,00:16
1,3.810568,3.741849,0.316211,00:16
2,3.736273,3.694241,0.321722,00:16
3,3.655913,3.656527,0.325893,00:16
4,3.576299,3.636308,0.329325,00:16
5,3.521190,3.626308,0.331152,00:16
6,3.477987,3.620083,0.331710,00:16
7,3.454979,3.621893,0.331376,00:16


In [64]:
learn_lm.save(f'{lang}fine_tuned_movie_reviews_bwd')
learn_lm.save_encoder(f'{lang}fine_tuned_enc_movie_reviews_bwd')

In [65]:
learn_lm = None
learn = None
gc.collect()

78

### Classifier

In [66]:
data_clas = (TextList.from_df(train_df, path, vocab=data_lm.vocab, cols=1)
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols=0)
    .databunch(bs=bs, num_workers=1, backwards=True))

data_clas.save(f'{lang}_textlist_class_movie_reviews_bwd')

In [67]:
data_clas = load_data(path, f'{lang}_textlist_class_movie_reviews_bwd', bs=bs, num_workers=1, backwards=True)

In [68]:
data_clas.classes, data_clas.c

(['negative', 'neutral', 'positive'], 3)

In [69]:
from sklearn.metrics import f1_score

@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1), average='micro')

mcc = MatthewsCorreff()

In [70]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1, mcc]).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc_movie_reviews_bwd')
learn_c.freeze()

In [71]:
lr=2e-2
lr *= bs/48

In [72]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,1.052091,1.150976,0.395161,0.395161,0.039934,00:07
1,0.995788,0.979934,0.451613,0.451613,0.157064,00:07


In [73]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.977321,0.966408,0.556452,0.556452,0.317506,00:07
1,0.937606,0.879384,0.604839,0.604839,0.417594,00:06


In [74]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.922581,0.986960,0.544355,0.544355,0.323410,00:09
1,0.833586,0.869632,0.629032,0.629032,0.439736,00:08


In [75]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.653352,1.386287,0.463710,0.463710,0.248490,00:11
1,0.586420,0.890083,0.596774,0.596774,0.384688,00:15


In [76]:
learn_c.unfreeze()
learn_c.fit_one_cycle(2, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.395189,0.982499,0.608871,0.608871,0.417089,00:16
1,0.362088,1.021848,0.596774,0.596774,0.396894,00:13


In [77]:
learn_c.save(f'{lang}clas_movie_reviews_bwd')

In [78]:
learn_c.load(f'{lang}clas_movie_reviews_bwd');

In [79]:
data_valid = (TextList.from_df(test_df, path, vocab=data_lm.vocab, cols=1)
    .split_none()
    .label_from_df(cols=0)
    .databunch(bs=bs, num_workers=1, backwards=True))

In [80]:
data_valid

TextClasDataBunch;

Train: LabelList (310 items)
x: TextList
xxbos काव्या अपनी xxunk और जिद में सिद्धार्थ को तबाह करने के लिए जिस्म को हथियार बनाती है ।,xxbos निर्माता : 
  फरहान अख्तर , रितेश सिधवानी 
 
  निर्देशक : 
  अभिनय देव 
 
  संगीत : 
  शंकर - अहसान - लॉय 
 
  कलाकार : 
  अभिषेक बच्चन , कंगना , xxunk जेन xxunk , जिमी शेरगिल , गौहर खान , शहाना गोस्वामी , बोमन ईरानी , अनुपम खेर 
 
  सेंसर सर्टिफिकेट : यू / ए * 2 घंटे 10 मिनट 
 
 
 
  थ्रिलर और मर्डर मिस्ट्री फिल्म बनाना हर किसी के बस की बात नहीं है । राज खुलने तक दर्शक को बाँधकर रखना और कातिल के चेहरे से परदा xxunk के बाद दर्शक को संतुष्ट करना कठिन काम है क्योंकि सभी को अपने प्रश्नों का ठोस उत्तर xxunk । इसके लिए कसी हुई स्क्रिप्ट और चुस्त निर्देशन की आवश्यकता होती है । 
 
  ‘ गेम ’ इन xxunk पर खरी नहीं उतरती और एक उबाऊ फिल्म के रूप में सामने आती है । हैरत इस बात की होती है कि इस फिल्म से फरहान अख्तर का नाम जुड़ा है जिनका बैनर अच्छी फिल्मों के लिए जाना जाता है । 
 
  फिल्म से जुड़े सारे लोगों का ध्यान केवल स्टाइलिश फिल्म बनाने क

In [81]:
%%time
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(test_df[1]), 'actual_label': list(test_df[0]), 'predicted_label': ['']*test_df.shape[0]}
all_nodes = list(set(train_df[0]))
for node in all_nodes:
    df_dict[node] = ['']*test_df.shape[0]
    
i2c = {}
for key, value in learn_c.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
for index, row in df_result.iterrows():
    pred = learn_c.predict(data_valid.train_ds[index])
    for node in all_nodes:
        row[node] = pred[2][learn_c.data.c2i[node]].item()
    row['predicted_label'] = i2c[pred[0].data.item()]
df_result.head()

CPU times: user 25.2 s, sys: 1.6 s, total: 26.8 s
Wall time: 24.6 s


,query,actual_label,predicted_label,positive,neutral,negative
0,काव्या अपनी खुन्नस और जिद में सिद्धार्थ को तबा...,negative,positive,0.724467,0.0532027,0.22233
1,"निर्माता :\nफरहान अख्तर, रितेश सिधवानी\n\nनिर्...",negative,negative,0.0010258,0.0754391,0.923535
2,असमर्पित रिश्ते में यकीन रखने वाले दोनों मौज-म...,positive,negative,0.221458,0.277527,0.501016
3,"बेवकूफियां को हबीब फैज़ल ने लिखा है, जिनका नाम...",neutral,negative,0.0394158,0.412733,0.547851
4,तब वह मोटी थी और जेरी ने उस पर ध्यान नहीं दिय...,neutral,negative,0.198066,0.0112672,0.790667


In [82]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 6 columns):
query              310 non-null object
actual_label       310 non-null object
predicted_label    310 non-null object
positive           310 non-null object
neutral            310 non-null object
negative           310 non-null object
dtypes: object(6)
memory usage: 14.7+ KB


In [83]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.6193548387096774

In [84]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.43682187918253335

In [85]:
f1_score(df_result['actual_label'], df_result['predicted_label'], average='micro')

0.6193548387096774